In [1]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import nltk
nltk.download('punkt_tab')      
nltk.download('wordnet')    
nltk.download('omw-1.4') 
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /home/emin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/emin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/emin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/emin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

1-Ai 0-Human

In [4]:
df = pd.read_csv("Training_Essay_Data.csv")
df

,text,generated
0,Car-free cities have become a subject of incre...,1
1,"Car Free Cities Car-free cities, a concept ga...",1
2,A Sustainable Urban Future Car-free cities ...,1
3,Pioneering Sustainable Urban Living In an e...,1
4,The Path to Sustainable Urban Living In an ...,1
...,...,...
29140,There has been a fuss about the Elector Colleg...,0
29141,Limiting car usage has many advantages. Such a...,0
29142,There's a new trend that has been developing f...,0
29143,As we all know cars are a big part of our soci...,0


In [5]:
english_basic_stop_word = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [6]:
lemmatizer = WordNetLemmatizer()

In [7]:
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
    filtered_words = [word for word in lemmatized_words if word not in english_basic_stop_word]
    return " ".join(filtered_words)

In [8]:
df["text"] = df["text"].apply(preprocess_text)
df

,text,generated
0,carfree city subject increasing debate year ur...,1
1,car free city carfree city concept gaining tra...,1
2,sustainable urban future carfree city emerging...,1
3,pioneering sustainable urban living era marked...,1
4,path sustainable urban living age rapid urbani...,1
...,...,...
29140,ha fuss elector college people confused work s...,0
29141,limiting car usage ha advantage putting lot po...,0
29142,trend ha developing year throttle affect mass ...,0
29143,car big society today car bigger impact people...,0


In [9]:
df["text"][0]

'carfree city subject increasing debate year urban area grapple challenge congestion pollution limited resource concept carfree city involves creating urban environment private automobile restricted completely banned focus alternative transportation method sustainable urban planning essay explores benefit challenge potential solution idea carfree city benefit carfree city environmental sustainability carfree city promote environmental sustainability reducing air pollution greenhouse gas emission fewer car road cleaner air decrease contribution global warming improved public health reduction automobile usage lead public health outcome fewer car road result fewer accident safer urban environment pedestrian cyclist air pollution lead reduced respiratory cardiovascular problem efficient space carfree city utilize urban space efficiently parking lot wide road repurposed green space park pedestrian zone enhancing quality life city reduced traffic congestion eliminating restricting car usage 

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
tokenizer.pad_token = tokenizer.eos_token

/home/emin/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
tokenizer.eos_token

'<|endoftext|>'

In [12]:
tokenizer.encode(df["text"][0])

[7718,
 5787,
 1748,
 2426,
 3649,
 4384,
 614,
 7876,
 1989,
 42146,
 4427,
 28014,
 12231,
 3614,
 8271,
 3721,
 1097,
 5787,
 1748,
 9018,
 4441,
 7876,
 2858,
 2839,
 27930,
 10770,
 3190,
 9301,
 2962,
 5559,
 9358,
 2446,
 13347,
 7876,
 5410,
 14268,
 25409,
 4414,
 4427,
 2785,
 4610,
 2126,
 1097,
 5787,
 1748,
 4414,
 1097,
 5787,
 1748,
 6142,
 26809,
 1097,
 5787,
 1748,
 7719,
 6142,
 26809,
 8868,
 1633,
 12231,
 16325,
 3623,
 25592,
 7380,
 1097,
 2975,
 21723,
 1633,
 10070,
 10156,
 3298,
 9917,
 6596,
 1171,
 1535,
 7741,
 27930,
 8748,
 1085,
 1171,
 1535,
 8055,
 7380,
 1097,
 2975,
 1255,
 7380,
 5778,
 14178,
 7876,
 2858,
 22382,
 34632,
 1633,
 12231,
 1085,
 5322,
 22949,
 21134,
 1917,
 6942,
 2272,
 1097,
 5787,
 1748,
 17624,
 7876,
 2272,
 18306,
 7647,
 1256,
 3094,
 2975,
 1128,
 333,
 29813,
 4077,
 2272,
 3952,
 22382,
 6516,
 27496,
 3081,
 1204,
 1748,
 5322,
 4979,
 28014,
 18591,
 28267,
 1097,
 8748,
 4979,
 28014,
 5322,
 3756,
 5443,
 31099,
 64

In [13]:
df["encoded_text"] = df["text"].apply(tokenizer.encode)
df

Token indices sequence length is longer than the specified maximum sequence length for this model (1633 > 1024). Running this sequence through the model will result in indexing errors


,text,generated,encoded_text
0,carfree city subject increasing debate year ur...,1,"[7718, 5787, 1748, 2426, 3649, 4384, 614, 7876..."
1,car free city carfree city concept gaining tra...,1,"[7718, 1479, 1748, 1097, 5787, 1748, 3721, 139..."
2,sustainable urban future carfree city emerging...,1,"[82, 24196, 7876, 2003, 1097, 5787, 1748, 1182..."
3,pioneering sustainable urban living era marked...,1,"[79, 7935, 1586, 13347, 7876, 2877, 6980, 7498..."
4,path sustainable urban living age rapid urbani...,1,"[6978, 13347, 7876, 2877, 2479, 5801, 7876, 16..."
...,...,...,...
29140,ha fuss elector college people confused work s...,0,"[3099, 34297, 43191, 4152, 661, 10416, 670, 11..."
29141,limiting car usage ha advantage putting lot po...,0,"[2475, 1780, 1097, 8748, 387, 4621, 5137, 1256..."
29142,trend ha developing year throttle affect mass ...,0,"[83, 10920, 387, 5922, 614, 29976, 2689, 2347,..."
29143,car big society today car bigger impact people...,0,"[7718, 1263, 3592, 1909, 1097, 5749, 2928, 661..."


In [14]:
seq_lengths = [len(seq) for seq in df['encoded_text']]
print(f"Mean: {np.mean(seq_lengths)}, Median: {np.median(seq_lengths)}")
maxlen = int(np.percentile(seq_lengths, 95))

Mean: 170.07335735117516, Median: 160.0


In [15]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, maxlen=546):
        self.texts = texts
        self.labels = labels
        self.maxlen = maxlen

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        text = torch.tensor(self.texts[idx][:self.maxlen], dtype=torch.long)
        label = torch.tensor(self.labels[idx], dtype=torch.float)
        return text, label

In [16]:
from keras.preprocessing.sequence import pad_sequences

2025-07-27 13:54:26.851739: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753613666.865064   18087 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753613666.869012   18087 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753613666.880494   18087 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753613666.880508   18087 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753613666.880510   18087 computation_placer.cc:177] computation placer alr

In [17]:
X = pad_sequences(df['encoded_text'], maxlen=maxlen, padding='post', truncating='post')
y = df['generated'].values

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Dataset ve DataLoader oluştur
train_dataset = TextDataset(X_train, y_train, maxlen=maxlen)
test_dataset = TextDataset(X_test, y_test, maxlen=maxlen)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [40]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=512, hidden_dim=64):
        super (TextClassifier, self).__init__()

        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True, num_layers=4)
        self.fc = nn.Linear(in_features=hidden_dim*2, out_features=1)
        self.dropout = nn.Dropout(p=0.7)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        last_hidden = lstm_out[:, -1, :]
        output = self.fc(self.dropout(last_hidden))

        return torch.sigmoid(output)
    
model = TextClassifier(vocab_size=tokenizer.vocab_size)
model

TextClassifier(
  (embedding): Embedding(50257, 512, padding_idx=0)
  (lstm): LSTM(512, 64, num_layers=4, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.7, inplace=False)
)

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device=device)
loss_fn = nn.BCELoss()
optimizer = optim.AdamW(params=model.parameters(), lr=1e-3)

In [42]:
def train_model(epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for text, label in train_loader:
            text, label = text.to(device=device), label.to(device=device)
            label = label.unsqueeze(1).float()


            optimizer.zero_grad()
            output = model(text)
            loss = loss_fn(output, label)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()   

        model.eval()
        correct = 0
        with torch.no_grad():
            for texts, labels in test_loader:
                texts, labels = texts.to(device), labels.to(device)
                preds = (model(texts).squeeze() > 0.5).float()
                correct += (preds == labels).sum().item()
        
        acc = correct / len(test_dataset)
        print(f'Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}, Accuracy = {acc:.4f}')

train_model(epochs=100)

Epoch 1: Loss = 0.5784, Accuracy = 0.8670
Epoch 2: Loss = 0.2743, Accuracy = 0.9499
Epoch 3: Loss = 0.1714, Accuracy = 0.9693
Epoch 4: Loss = 0.1184, Accuracy = 0.9708
Epoch 5: Loss = 0.1048, Accuracy = 0.9798
Epoch 6: Loss = 0.0741, Accuracy = 0.9825
Epoch 7: Loss = 0.0490, Accuracy = 0.9825
Epoch 8: Loss = 0.0415, Accuracy = 0.9842
Epoch 9: Loss = 0.0324, Accuracy = 0.9878
Epoch 10: Loss = 0.0241, Accuracy = 0.9876
Epoch 11: Loss = 0.0218, Accuracy = 0.9880
Epoch 12: Loss = 0.0178, Accuracy = 0.9899
Epoch 13: Loss = 0.0193, Accuracy = 0.9907
Epoch 14: Loss = 0.0150, Accuracy = 0.9897
Epoch 15: Loss = 0.0097, Accuracy = 0.9900
Epoch 16: Loss = 0.0103, Accuracy = 0.9870
Epoch 17: Loss = 0.0081, Accuracy = 0.9902
Epoch 18: Loss = 0.0066, Accuracy = 0.9892
Epoch 19: Loss = 0.0078, Accuracy = 0.9916
Epoch 20: Loss = 0.0076, Accuracy = 0.9899
Epoch 21: Loss = 0.0121, Accuracy = 0.9894
Epoch 22: Loss = 0.0102, Accuracy = 0.9878
Epoch 23: Loss = 0.0056, Accuracy = 0.9921
Epoch 24: Loss = 0.0

In [43]:
from sklearn.metrics import classification_report

def evaluate():
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for texts, labels in test_loader:
            texts = texts.to(device)
            preds = (model(texts).squeeze() > 0.5).int().cpu()
            y_true.extend(labels.int().tolist())
            y_pred.extend(preds.tolist())
    print(classification_report(y_true, y_pred))

evaluate()

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3539
           1       0.99      0.99      0.99      2290

    accuracy                           0.99      5829
   macro avg       0.99      0.99      0.99      5829
weighted avg       0.99      0.99      0.99      5829



In [48]:
def predict(text, model, tokenizer, device='cpu'):
    # 1. Metni tokenize et (input_ids tensor'ı elde edilir)
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids'].to(device)

    # 2. Modeli değerlendirme moduna al ve tahmin yap
    model.to(device)
    model.eval()
    with torch.no_grad():
        output = model(input_ids)       # Çıktı: sigmoid uygulanmış [0,1] arası tek değer
        prob = output.item()            # Bu değer, metnin "AI tarafından yazılmış olma olasılığıdır"

    # 3. Sınıf belirleme
    label = "AI" if prob >= 0.5 else "Human"

    # 4. Tahmini, ait olduğu sınıfa göre yorumla:
    confidence = prob if label == "AI" else (1 - prob)

    return {
        "text": text,
        "label": label,
        "confidence": confidence  # Modelin bu sınıfa olan güveni (0-1)
    }

# Örnek kullanım
sentence = "The old lighthouse stood sentinel on the craggy cliffs, its beam sweeping across the turbulent sea. Generations of keepers had climbed its winding stairs, their lives dedicated to guiding ships safely through treacherous waters. Though automated now, a sense of enduring vigilance still clung to its weathered stone walls, a silent testament to countless storms weathered and lives saved."
result = predict(sentence, model, tokenizer, device=device)

print(f"Text: {result['text']}")
print(f"Guess: {result['label']} (%{result['confidence']*100:.2f} confidence)")


Text: The old lighthouse stood sentinel on the craggy cliffs, its beam sweeping across the turbulent sea. Generations of keepers had climbed its winding stairs, their lives dedicated to guiding ships safely through treacherous waters. Though automated now, a sense of enduring vigilance still clung to its weathered stone walls, a silent testament to countless storms weathered and lives saved.
Guess: AI (%100.00 confidence)
